# Recommender Systems Lab: Building a Movie Recommendation Engine

## Lab Overview

In this hands-on session, we'll build a complete movie recommendation system using real data. You'll implement multiple algorithms, evaluate their performance, and understand the business implications of different approaches.

**Business Context**: You're consulting for a streaming service looking to increase user engagement through personalized recommendations.

---

## Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# For data processing and algorithms
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from collections import defaultdict
import urllib.request
import zipfile
import os

print("✅ Setup complete! Let's build some recommender systems!")

---

## Part 1: Data Loading and Exploration

In this section, we'll load the MovieLens dataset and conduct exploratory data analysis to understand user behavior patterns, movie popularity distributions, and data quality. This foundational analysis will inform our recommendation strategy and help identify key business insights about user engagement.

#### 1.1 Load the MovieLens Dataset

In [ ]:
# Load the MovieLens 100K dataset with movie information
# Download and extract the dataset
url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
urllib.request.urlretrieve(url, "movielens.zip")

with zipfile.ZipFile("movielens.zip", "r") as zip_ref:
    zip_ref.extractall("movielens")

# Load ratings data
ratings_df = pd.read_csv('movielens/ml-100k/u.data', sep='\t',
                        names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')

# Load movie information for content-based filtering
movies_df = pd.read_csv('movielens/ml-100k/u.item', sep='|', encoding='latin-1',
                       names=['item_id', 'title', 'release_date', 'video_release_date',
                             'imdb_url', 'unknown', 'Action', 'Adventure', 'Animation',
                             'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
                             'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                             'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

# Extract genres
genre_columns = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
                'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

print(f"📊 Dataset shape: {ratings_df.shape}")
print(f"👥 Number of users: {ratings_df['user_id'].nunique()}")
print(f"🎬 Number of movies: {ratings_df['item_id'].nunique()}")
print(f"⭐ Rating scale: {ratings_df['rating'].min()} to {ratings_df['rating'].max()}")
print(f"📅 Date range: {ratings_df['timestamp'].min()} to {ratings_df['timestamp'].max()}")
print(f"\n🏷️ Number of genres: {len(genre_columns)}")

#### 1.2 Exploratory Data Analysis

In [ ]:
# TODO: Complete the visualization code

# 1. Create a bar chart showing how many times each rating value (1-5) appears in the dataset
# YOUR CODE HERE

# 2. Create a histogram showing the distribution of how many ratings each user has made
# YOUR CODE HERE

# 3. Create a line plot showing movies ranked by popularity (number of ratings received)
# YOUR CODE HERE

# 4. Create a bar chart showing how many movies belong to each genre category
# YOUR CODE HERE


#### 💡 Business Insight Questions

In [ ]:
# TODO: Let's analyze some business-relevant patterns

# 1. What percentage of users are "power users" (>100 ratings)?
# YOUR CODE HERE
print(f"👤 Power users (>100 ratings): {}%")

# 2. What percentage of movies have very few ratings (<10)?
# YOUR CODE HERE
print(f"🎬 Rare movies (<10 ratings): {}%")

# 3. Calculate the percentage of ratings that come from the top 20% most active users
# This demonstrates the 80/20 rule in user engagement
# YOUR CODE HERE
print(f"📊 Percentage of ratings from top 20% users: {}%")

## Part 2: Building Baseline Recommenders

Now we'll implement our first recommendation algorithms, starting with simple but effective baseline approaches. These methods provide a foundation for comparison and handle common challenges like cold start problems for new users.

#### 2.1 Popularity-Based Recommender

In [ ]:
class PopularityRecommender:
    """
    Simple recommender that suggests the most popular movies.
    Good baseline and handles cold start for new users.
    """
    def __init__(self, min_ratings=50):
        self.min_ratings = min_ratings
        self.popular_movies = None

    def fit(self, ratings_df):
        # TODO: Implement `fit` method for `PopularityRecommender`
        # Steps:
        # - Filter movies with minimum ratings
        # - Compute movie stats, e.g., mean rating, number of ratings, etc.
        # - Combine movie stats into a popularity score
        # - Save in `self.popular_movies` the movies ordered by popularity score
        #     Note that the `recommend_for_user` method selects the top-n movies from `self.popular_movies`

        # YOUR CODE HERE

        # Final step
        self.popular_movies = ...
        return self

    def recommend_for_user(self, user_id=None, n=10):
        """Return top N popular movies (user_id is ignored for popularity-based)"""
        return self.popular_movies.head(n)

In [ ]:
def add_movie_titles(recommendations_df, movies_df):
    """Helper function to add movie titles to recommendations"""
    return recommendations_df.merge(movies_df[['item_id', 'title']], on='item_id', how='left')

Generate recommendations with the popularity recommender

In [ ]:
pop_rec = PopularityRecommender(min_ratings=50)
pop_rec.fit(ratings_df)

print("🏆 Top 10 Popular Movies:")
popular_recommendations = pop_rec.recommend_for_user(n=10)

# Add movie titles
popular_with_titles = add_movie_titles(popular_recommendations, movies_df)
print(popular_with_titles)

---

## Part 3: Collaborative Filtering

In this section, we'll build recommenders based on collaborative filtering, namely user-based collaborative filtering and a more sophisticated approach using matrix factorization (SVD) to discover hidden patterns in user preferences. SVD can uncover latent factors that explain user-item interactions and often provides superior personalization compared to simpler methods.

#### 3.1 User-Based Collaborative Filtering

In [ ]:
class UserBasedCFRecommender:
    def __init__(self, n_similar_users=50):
        self.n_similar_users = n_similar_users
        self.user_item_matrix = None

    def fit(self, ratings_df):
        """Build user-item matrix from ratings data"""
        self.user_item_matrix = ratings_df.pivot_table(
            index='user_id',
            columns='item_id',
            values='rating'
        ).fillna(0)
        return self

    def recommend_for_user(self, user_id, n=10):
        """
        Recommend movies based on similar users' preferences
        """
        if user_id not in self.user_item_matrix.index:
            return pd.DataFrame(columns=['item_id', 'predicted_score'])

        # Get the target user's ratings
        target_user_ratings = self.user_item_matrix.loc[user_id]

        # TODO: Calculate cosine similarity between target user and all other users
        # Hint: Use cosine_similarity from sklearn with proper reshaping
        # Expected: user_similarities should be a 1D array of similarity scores
        # YOUR CODE HERE
        user_similarities = ...

        # Get indices of most similar users (excluding the user themselves)
        similar_users_indices = np.argsort(user_similarities)[::-1][1:self.n_similar_users+1]
        similar_users = self.user_item_matrix.index[similar_users_indices]

        # Generate recommendations based on what similar users liked
        # but the target user hasn't seen
        recommendations = defaultdict(float)
        for idx, similar_user in enumerate(similar_users):
            similarity = user_similarities[similar_users_indices[idx]]
            similar_user_ratings = self.user_item_matrix.loc[similar_user]

            # Find movies the similar user rated highly but target user hasn't seen
            for movie_id, rating in similar_user_ratings.items():
                if target_user_ratings[movie_id] == 0 and rating >= 4:
                    recommendations[movie_id] += rating * similarity

        # Sort and return top N
        top_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n]
        return pd.DataFrame(top_recommendations, columns=['item_id', 'predicted_score'])

Generate recommendations with the user-based CF recommender

In [ ]:
cf_recommender = UserBasedCFRecommender(n_similar_users=50)
cf_recommender.fit(ratings_df)

# Test with a sample user
sample_user = ratings_df['user_id'].sample(1).iloc[0]
print(f"👤 Recommendations for User {sample_user}:")

# Get recommendations using the class method
user_cf_recs = cf_recommender.recommend_for_user(sample_user, n=10)
# Add movie titles
user_cf_recs_with_titles = add_movie_titles(user_cf_recs, movies_df)
print(user_cf_recs_with_titles)

### 3.2 Matrix Factorization with SVD

In [ ]:
class SVDRecommender:
    """
    Efficient SVD-based recommender using scipy's optimized implementation
    """
    def __init__(self, n_factors=50):
        self.n_factors = n_factors
        self.user_item_matrix = None
        self.predicted_ratings = None
        self.user_means = None
        self.user_ids = None
        self.item_ids = None

    def fit(self, train_df):
        """Train SVD model and store reference data"""
        self.ratings_df = train_df

        # Create user-item matrix
        self.user_item_matrix = train_df.pivot_table(
            index='user_id',
            columns='item_id',
            values='rating'
        )

        # Store indices for lookup
        self.user_ids = self.user_item_matrix.index
        self.item_ids = self.user_item_matrix.columns

        # Fill NaN and normalize
        matrix_filled = self.user_item_matrix.fillna(0)
        self.user_means = np.mean(matrix_filled.values, axis=1)
        matrix_normalized = matrix_filled.values - self.user_means.reshape(-1, 1)

        # Apply SVD
        U, sigma, Vt = svds(csr_matrix(matrix_normalized), k=self.n_factors)
        sigma = np.diag(sigma)

        # Reconstruct ratings matrix
        self.predicted_ratings = np.dot(np.dot(U, sigma), Vt) + self.user_means.reshape(-1, 1)

        return self

    def predict(self, user_id, item_id):
        """Predict rating for user-item pair(s)"""
        if isinstance(user_id, (list, np.ndarray)):
            return [self._predict_single(u, i) for u, i in zip(user_id, item_id)]
        else:
            return self._predict_single(user_id, item_id)

    def _predict_single(self, user_id, item_id):
        """Predict rating for single user-item pair"""
        try:
            user_idx = self.user_ids.get_loc(user_id)
            item_idx = self.item_ids.get_loc(item_id)
            prediction = self.predicted_ratings[user_idx, item_idx]
            return np.clip(prediction, 1, 5)
        except KeyError:
            # Return global mean for unknown users/items
            return np.mean(self.user_means) + 3  # Reasonable default

    def recommend_for_user(self, user_id, n=10):
        """Get top N recommendations for a user"""
        if user_id not in self.user_ids:
            return pd.DataFrame(columns=['item_id', 'predicted_rating', 'title'])

        # Get user index
        user_idx = self.user_ids.get_loc(user_id)
        user_predictions = self.predicted_ratings[user_idx]

        # Get items the user has already rated (need to store ratings_df)
        if hasattr(self, 'ratings_df'):
            user_items = self.ratings_df[self.ratings_df['user_id'] == user_id]['item_id'].values
        else:
            user_items = []

        # Create recommendations
        recommendations = []
        for idx, item_id in enumerate(self.item_ids):
            if item_id not in user_items:
                recommendations.append((item_id, user_predictions[idx]))

        # Sort and return top N
        recommendations.sort(key=lambda x: x[1], reverse=True)
        rec_df = pd.DataFrame(recommendations[:n], columns=['item_id', 'predicted_rating'])
        return rec_df

Generate recommendations with the SVD recommender

In [ ]:
# Usage
print("🔧 Training SVD model...")
train_data = ratings_df.sample(frac=0.8, random_state=42)
test_data = ratings_df.drop(train_data.index)

svd_model = SVDRecommender(n_factors=50)
svd_model.fit(train_data)
print("✅ SVD model trained!")

# Test recommendations
test_user = ratings_df['user_id'].sample(1).iloc[0]
print(f"🎯 SVD Recommendations for User {test_user}:")

svd_recommendations = svd_model.recommend_for_user(test_user, n=10)
svd_recommendations = add_movie_titles(svd_recommendations, movies_df)

print(svd_recommendations)

---

## Part 4: Content-Based Filtering

In this section, we'll build recommenders that leverage movie characteristics like genres and release year to find similar content. Content-based approaches excel at recommending items similar to what users have already enjoyed and can handle new items effectively.

#### 4.1 Feature Engineering for Movies

In [ ]:
# Prepare content features for movies
def prepare_movie_features(movies_df):
    """
    Create feature vectors for movies based on genres and other attributes
    """
    movies_df = movies_df.copy()
    # Genre features (already binary)
    genre_features = movies_df[genre_columns].values

    # Extract year from release date
    movies_df['year'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year
    movies_df['year'].fillna(movies_df['year'].median(), inplace=True)

    # Normalize year to 0-1 range
    min_year = movies_df['year'].min()
    max_year = movies_df['year'].max()
    movies_df['year_normalized'] = (movies_df['year'] - min_year) / (max_year - min_year)

    # Combine all features
    feature_matrix = np.column_stack([
        genre_features,
        movies_df['year_normalized'].values.reshape(-1, 1)
    ])

    return feature_matrix, movies_df

# Prepare features
movie_features, movies_enhanced = prepare_movie_features(movies_df)
print(f"📐 Movie feature matrix shape: {movie_features.shape}")
print(f"📊 Features include: {len(genre_columns)} genres + 1 year feature")

#### 4.2 Content-Based Item Similarity

In [ ]:
class ContentBasedRecommender:
    def __init__(self, movies_df, movie_features):
        self.movies_df = movies_df
        self.movie_features = movie_features
        self.item_similarities = None
        self.ratings_df = None

    def fit(self, ratings_df=None):
        """Calculate item-item similarities and store ratings data"""
        if ratings_df is not None:
            self.ratings_df = ratings_df

        # TODO: Compute item similarities between all movies
        # Hint: Use cosine similarity between `self.movie_features`, and store the result in `self.item_similarities`
        # YOUR CODE HERE
        return self

    def get_similar_items(self, item_id, n=10):
        """Find n most similar items to the given item"""
        if item_id not in self.movies_df['item_id'].values:
            return pd.DataFrame(columns=['item_id', 'title', 'similarity_score'])

        # Get index of the movie
        idx = self.movies_df[self.movies_df['item_id'] == item_id].index[0]

        # Get similarity scores and sort
        sim_scores = list(enumerate(self.item_similarities[idx]))
        sim_scores.sort(key=lambda x: x[1], reverse=True)

        # Get top N similar movies (excluding itself)
        similar_indices = [i[0] for i in sim_scores[1:n+1]]
        similar_movies = self.movies_df.iloc[similar_indices][['item_id', 'title']].copy()
        similar_movies['similarity_score'] = [i[1] for i in sim_scores[1:n+1]]

        return similar_movies

    def recommend_for_user(self, user_id, n=10):
        """Recommend items for a user (now uses stored ratings_df)"""
        if self.ratings_df is None:
            raise ValueError("No ratings data available. Call fit(ratings_df) first.")

        # Get user's rated movies
        user_movies = self.ratings_df[self.ratings_df['user_id'] == user_id]
        if len(user_movies) == 0:
            return pd.DataFrame(columns=['item_id', 'content_score', 'title'])

        # Get highly rated movies by the user
        liked_movies = user_movies[user_movies['rating'] >= 4]['item_id'].values
        if len(liked_movies) == 0:
            liked_movies = user_movies.nlargest(3, 'rating')['item_id'].values

        # Find similar movies to what the user liked
        # Aggregate similarity scores across all liked movies
        recommendations = defaultdict(float)

        for movie_id in liked_movies:
            if movie_id in self.movies_df['item_id'].values:
                similar_movies = self.get_similar_items(movie_id, n=20)
                for _, row in similar_movies.iterrows():
                    # Don't recommend movies the user has already seen
                    if row['item_id'] not in user_movies['item_id'].values:
                        recommendations[row['item_id']] += row['similarity_score']

        # Sort and get top N
        top_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n]
        result_df = pd.DataFrame(top_recommendations, columns=['item_id', 'content_score'])
        result_df = result_df.merge(self.movies_df[['item_id', 'title']], on='item_id')

        return result_df


Generate recommendations with the content-based recommender

In [ ]:
# Create and test content-based recommender
content_rec = ContentBasedRecommender(movies_enhanced, movie_features)
content_rec.fit(ratings_df)

In [ ]:
# Test: Find similar movies to Star Wars
star_wars_id = movies_df[movies_df['title'].str.contains('Star Wars', case=False)]['item_id'].iloc[0]
print(f"🎬 Movies similar to '{movies_df[movies_df['item_id']==star_wars_id]['title'].iloc[0]}':")
print(content_rec.get_similar_items(star_wars_id, n=10))

In [ ]:
# Test: Recommendations for a user
print(f"\n📚 Content-based recommendations for User {sample_user}:")
print(content_rec.recommend_for_user(sample_user, n=10))

#### 4.3 Analyzing Content Patterns

In [ ]:
# Visualize content similarity
def visualize_genre_similarity(content_rec, sample_size=50):
    """Visualize similarity between movies based on content"""
    # Sample movies for visualization
    sample_indices = np.random.choice(len(movies_df), sample_size, replace=False)
    sample_similarities = content_rec.item_similarities[sample_indices][:, sample_indices]

    # Create labels with movie titles (truncated)
    sample_movies = movies_df.iloc[sample_indices]
    labels = [title[:20] + '...' if len(title) > 20 else title
              for title in sample_movies['title'].values]

    # Plot heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(sample_similarities, cmap='YlOrRd',
                xticklabels=labels, yticklabels=labels,
                cbar_kws={'label': 'Content Similarity'})
    plt.title('Content Similarity Matrix (Sample of Movies)')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

visualize_genre_similarity(content_rec, sample_size=30)

---

## Part 5: Evaluation with Precision and Recall

Proper evaluation is crucial for understanding which algorithms work best for your specific use case. In this section, we'll implement standard recommendation metrics and compare our different approaches to determine their relative strengths and weaknesses.

#### 5.1 Implementing Precision@K and Recall@K

In [ ]:
def precision_recall_at_k(actual_items, predicted_items, k=10):
    """
    Calculate precision and recall at k for a single user
    """
    predicted_items_at_k = predicted_items[:k]

    # TODO: Implement Precision@K
    # YOUR CODE HERE
    precision = ...

    # TODO: Implement Recall@K
    # YOUR CODE HERE
    recall = ...

    return precision, recall

def evaluate_recommender_precision_recall(recommender, test_data, k_values=[5, 10, 20]):
    """Evaluate a recommender system using precision and recall at different K values"""
    results = {}
    test_users = test_data['user_id'].unique()[:100]  # Sample for speed

    for k in k_values:
        precisions, recalls = [], []

        for user_id in test_users:
            # Get user's highly-rated test items
            user_test_items = test_data[
                (test_data['user_id'] == user_id) & (test_data['rating'] >= 4)
            ]['item_id'].values

            if len(user_test_items) == 0:
                continue

            # Get recommendations
            try:
                recommendations = recommender.recommend_for_user(user_id, n=k)
                predicted_items = recommendations['item_id'].values if len(recommendations) > 0 else []
            except:
                continue

            # Calculate metrics
            precision, recall = precision_recall_at_k(user_test_items, predicted_items, k)
            precisions.append(precision)
            recalls.append(recall)

        # Store results
        if precisions:
            results[f'precision@{k}'] = np.mean(precisions)
            results[f'recall@{k}'] = np.mean(recalls)
            prec, rec = results[f'precision@{k}'], results[f'recall@{k}']
            results[f'f1@{k}'] = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
        else:
            results[f'precision@{k}'] = results[f'recall@{k}'] = results[f'f1@{k}'] = 0.0

    return results

In [ ]:
# Re-train models on train data
pop_rec = PopularityRecommender(min_ratings=50).fit(train_data)
content_rec = ContentBasedRecommender(movies_enhanced, movie_features).fit(train_data)
cf_rec = UserBasedCFRecommender(n_similar_users=50).fit(train_data)
svd_rec = SVDRecommender(n_factors=50).fit(train_data)

print("✅ All models trained!")

In [ ]:
print("📊 Evaluating algorithms with Precision@K and Recall@K:\n")

algorithms = {
    'Popularity-Based': pop_rec,
    'Content-Based': content_rec,
    'User-Based CF': cf_rec,
    'SVD (Matrix Factorization)': svd_rec
}

evaluation_results = {}
for name, recommender in algorithms.items():
    print(f"Evaluating {name}...")
    results = evaluate_recommender_precision_recall(recommender, test_data, k_values=[5, 10, 20])
    evaluation_results[name] = results
    print(f"{name}: Precision@10={results['precision@10']:.3f}, Recall@10={results['recall@10']:.3f}")

#### 5.2 Comprehensive Evaluation Framework

In [ ]:
def evaluate_recommender_comprehensive(recommender, test_data, k_values=[5, 10, 20]):
    """
    Evaluate recommender on multiple metrics and different K values
    """
    results = defaultdict(dict)

    # Get unique users in test set
    test_users = test_data['user_id'].unique()[:100]  # Sample for speed

    for k in k_values:
        all_precisions = []
        all_recalls = []
        recommended_items = set()

        for user_id in test_users:
            # Get user's test items (items they rated highly)
            user_test_items = test_data[
                (test_data['user_id'] == user_id) &
                (test_data['rating'] >= 4)
            ]['item_id'].values

            if len(user_test_items) == 0:
                continue

            # Get recommendations
            try:
                recs = recommender.recommend_for_user(user_id, n=k)
                rec_items = recs['item_id'].values
            except Exception as e:
                # Skip if recommender fails (e.g., user not in training data)
                continue

            # Calculate metrics
            hits = len(set(rec_items) & set(user_test_items))
            precision = hits / k if k > 0 else 0
            recall = hits / len(user_test_items) if len(user_test_items) > 0 else 0

            all_precisions.append(precision)
            all_recalls.append(recall)
            recommended_items.update(rec_items)

        # Store results
        results[k]['precision'] = np.mean(all_precisions) if all_precisions else 0
        results[k]['recall'] = np.mean(all_recalls) if all_recalls else 0
        results[k]['f1'] = 2 * results[k]['precision'] * results[k]['recall'] / \
                          (results[k]['precision'] + results[k]['recall']) \
                          if (results[k]['precision'] + results[k]['recall']) > 0 else 0
        results[k]['coverage'] = len(recommended_items) / len(test_data['item_id'].unique())

    return dict(results)

In [ ]:
# Evaluate all recommenders
print("\n📊 Comprehensive Evaluation Results:\n")

# Use your trained recommenders with unified interface
recommenders = {
    'Popularity': pop_rec,
    'Content-Based': content_rec,
    'CF User-Based': cf_rec,
    'SVD Matrix Factorization': svd_rec
}

for name, rec in recommenders.items():
    print(f"\n{name} Recommender:")
    results = evaluate_recommender_comprehensive(rec, test_data)

    # Display results in a nice table
    metrics_df = pd.DataFrame(results).T
    metrics_df.index.name = 'K'
    print(metrics_df.round(3))

#### 5.3 Visualizing Performance Trade-offs

In [ ]:
def plot_precision_recall_curves(recommenders_results):
    """
    Plot precision-recall curves for different recommenders
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot 1: Precision and Recall vs K
    k_values = [5, 10, 20]

    # Use the evaluation results we already have
    for name, results in evaluation_results.items():
        precisions = [results[f'precision@{k}'] for k in k_values]
        recalls = [results[f'recall@{k}'] for k in k_values]

        ax1.plot(k_values, precisions, marker='o', label=f'{name} (Precision)')
        ax1.plot(k_values, recalls, marker='s', linestyle='--', label=f'{name} (Recall)')

    ax1.set_xlabel('K (Number of Recommendations)')
    ax1.set_ylabel('Score')
    ax1.set_title('Precision and Recall vs K')
    ax1.legend()
    ax1.grid(True)

    # Plot 2: Precision vs Recall trade-off
    for name, results in evaluation_results.items():
        k_values_extended = [5, 10, 15, 20]
        precisions = []
        recalls = []

        for k in k_values_extended:
            if f'precision@{k}' in results:
                precisions.append(results[f'precision@{k}'])
                recalls.append(results[f'recall@{k}'])

        ax2.plot(recalls, precisions, marker='o', label=name)

    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Trade-off')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_precision_recall_curves(evaluation_results)

---

## Part 6: Production Considerations

Moving beyond algorithmic metrics, this section focuses on preparing systems for production deployment. We'll build a production-ready service with proper error handling and monitoring.

#### 6.1 Production Deployment Considerations


In [ ]:
class RecommendationService:
    """
    Production-ready recommendation service with caching and monitoring
    """
    def __init__(self, models_dict, movies_df):
        self.models = models_dict
        self.movies_df = movies_df
        self.cache = {}
        self.metrics = defaultdict(int)

    def get_recommendations(self, user_id, algorithm='content', n=10, use_cache=True):
        """
        Get recommendations with fallback strategies
        """
        start_time = datetime.now()

        # TODO: Implement caching, if `use_cache` is True
        # 1. Create cache key from user_id, algorithm, and n
        # 2. Check if key exists in self.cache
        # 3. Update cache hit/miss metrics, under `cache['cache_hits']` and `cache['cache_misses']`
        # 4. Return cached result if available
        # YOUR CODE HERE

        # If result not available in cache, compute recommendations
        try:
            if algorithm in self.models:
                recs = self.models[algorithm].recommend_for_user(user_id, n=n)
            else:
                raise ValueError(f"Unknown algorithm: {algorithm}")
        except Exception as e:
            # Fallback to popularity
            self.metrics['fallbacks'] += 1
            recs = self.models['popularity'].recommend_for_user(user_id, n=n)

        # Add movie titles if not present
        if len(recs) > 0:
            recs = add_movie_titles(recs, movies_df)

        # TODO: Cache computed recommendations, if `use_cache` is True
        # YOUR CODE HERE

        # Record metrics
        response_time = (datetime.now() - start_time).total_seconds()
        self.metrics['total_requests'] += 1
        self.metrics['avg_response_time'] = (
            (self.metrics['avg_response_time'] * (self.metrics['total_requests'] - 1) + response_time)
            / self.metrics['total_requests']
        )

        return recs

    def get_metrics_summary(self):
        """Get service metrics"""
        return {
            'total_requests': self.metrics['total_requests'],
            'cache_hit_rate': self.metrics['cache_hits'] / self.metrics['total_requests']
                             if self.metrics['total_requests'] > 0 else 0,
            'fallback_rate': self.metrics['fallbacks'] / self.metrics['total_requests']
                            if self.metrics['total_requests'] > 0 else 0,
            'avg_response_time_ms': self.metrics['avg_response_time'] * 1000
        }

In [ ]:
service = RecommendationService({
    'content': content_rec,
    'popularity': pop_rec
}, movies_df)

# Simulate production usage
print("\n🚀 Simulating Production Usage:\n")

# Different user scenarios
test_scenarios = [
    {'user_id': 1, 'algorithm': 'content'},     # Normal user
    {'user_id': 1, 'algorithm': 'content'},     # Cached request
    {'user_id': 50, 'algorithm': 'popularity'}, # Popularity-based
    {'user_id': 9999, 'algorithm': 'content'},  # New user (will fallback)
]

for scenario in test_scenarios:
    recs = service.get_recommendations(**scenario)
    print(f"User {scenario['user_id']} ({scenario['algorithm']}): {len(recs)} recommendations")

print("\n📊 Service Metrics:")
for metric, value in service.get_metrics_summary().items():
    if 'rate' in metric:
        print(f"  {metric}: {value:.2%}")
    elif 'time' in metric:
        print(f"  {metric}: {value:.2f}")
    else:
        print(f"  {metric}: {value}")

## Part 7: Key Takeaways and Discussion

Finally, we'll synthesize our findings into actionable business recommendations and strategic insights. This section provides a framework for choosing the right approach for different scenarios and discusses important considerations for real-world implementation.

#### 7.1 Algorithm Comparison Summary

In [ ]:
# Create a comprehensive comparison
print("🎯 ALGORITHM COMPARISON SUMMARY:\n")

comparison_data = {
    'Algorithm': ['Popularity', 'Content-Based', 'User-Based CF', 'SVD (Matrix Factorization)'],
    'Cold Start (New Users)': ['✅ Excellent', '✅ Excellent', '❌ Poor', '❌ Poor'],
    'Cold Start (New Items)': ['❌ Poor', '✅ Excellent', '❌ Poor', '❌ Poor'],
    'Scalability': ['✅ Excellent', '✅ Good', '⚠️ Moderate', '✅ Good'],
    'Interpretability': ['✅ High', '✅ High', '⚠️ Moderate', '❌ Low'],
    'Personalization': ['❌ None', '⚠️ Moderate', '✅ High', '✅ High'],
    'Data Requirements': ['Low', 'Item Features', 'User-Item Ratings', 'User-Item Ratings']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

#### 7.2 Business Recommendations


In [ ]:
print("\n📋 KEY BUSINESS RECOMMENDATIONS:\n")

recommendations = [
    "1. 🚀 Start Simple: Begin with popularity-based recommendations as a baseline",
    "2. 🎯 Hybrid Approach: Combine content-based for new items with CF for personalization",
    "3. 📊 Measure Everything: Track both algorithmic metrics (precision/recall) and business KPIs",
    "4. 🔄 Continuous Improvement: A/B test different algorithms and parameters",
    "5. ⚡ Performance Matters: Cache recommendations and pre-compute when possible",
    "6. 🎨 Diversity is Key: Don't just optimize for accuracy - consider novelty and diversity",
    "7. 🛡️ Plan for Failures: Always have a fallback strategy (e.g., popularity)"
]

for rec in recommendations:
    print(rec)

print("\n💭 DISCUSSION QUESTIONS:")
questions = [
    "- How would you handle recommendations for a brand new streaming service with no data?",
    "- What additional features could improve content-based recommendations?",
    "- How do you balance exploitation (recommending sure hits) vs exploration (discovering new content)?",
    "- How would you detect and mitigate filter bubbles?"
]

for question in questions:
    print(question)

#### 🏁 Lab Complete!


In [ ]:
print("""
🎉 Congratulations! You've built a complete recommendation system!

✅ What you've accomplished:
- Analyzed real user behavior data
- Implemented popularity-based, content-based, and collaborative filtering
- Evaluated algorithms using precision and recall metrics
- Considered production deployment challenges
- Analyzed business impact

📚 Next steps:
1. Try combining content-based and collaborative filtering in a hybrid approach
2. Experiment with deep learning methods (neural collaborative filtering)
3. Add contextual features (time of day, device type)
4. Implement online learning for real-time adaptation
""")